<a href="https://colab.research.google.com/github/jinhuioh/hi_ai_himedia/blob/master/%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81_%EC%9C%A0%EC%82%AC%EB%8F%84_%EC%B8%A1%EC%A0%95(%EC%BD%94%EC%82%AC%EC%9D%B8%EC%9C%A0%EC%82%AC%EB%8F%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import mlxtend #apiori알고리즘 관련 라이브러리
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

from mlxtend.preprocessing import TransactionEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [2]:

data = np.array([
    ['우유', '기저귀', '쥬스'],
    ['양상추', '기저귀', '맥주'],
    ['우유', '양상추', '기저귀', '쥬스'],
    ['양상추', '맥주']
])

In [4]:

te = TransactionEncoder()

In [5]:
#2차행렬 T/F로 인코딩하고 바꿔주기.
te_ary = te.fit(data).transform(data)
te_ary

array([[ True, False, False,  True,  True],
       [ True,  True,  True, False, False],
       [ True, False,  True,  True,  True],
       [False,  True,  True, False, False]])

In [6]:
df = pd.DataFrame(te_ary, columns=te.columns_)

In [7]:
df

,기저귀,맥주,양상추,우유,쥬스
0,True,False,False,True,True
1,True,True,True,False,False
2,True,False,True,True,True
3,False,True,True,False,False


In [8]:
#빈도수를 가지고 지지도를 그려보자.
from mlxtend.frequent_patterns import apriori

In [32]:
#지지도 0.5이상인 행만 추출
result = apriori(df, min_support=0.5, use_colnames=True)
result

,support,itemsets
0,0.75,(기저귀)
1,0.50,(맥주)
2,0.75,(양상추)
3,0.50,(우유)
4,0.50,(쥬스)
5,0.50,"(기저귀, 양상추)"
6,0.50,"(기저귀, 우유)"
7,0.50,"(기저귀, 쥬스)"
8,0.50,"(맥주, 양상추)"
9,0.50,"(쥬스, 우유)"


In [33]:
type(result)

pandas.core.frame.DataFrame

In [10]:
##쥬스+우유를 함께 산 사람에게 추천을 해준다면
##맥주보다는 기저귀 추천해야함.
##지지도 기반으로 판단!!!!

In [11]:
#one-hot인코딩 방식의 테이블 필요!
#실제로는 물건수가 아주 많음 => 0이 많은 2차원 행렬로 메모리소요 엄청남,시간오래걸림.
#item 수 => 약380개(열)
#유저 수 => 800개(행)
# 0이 2차원 데이터 : 희소행렬(sparse matrix)
#내가 산 물건을 다른안산사람들에게 아이템기반 추천.
#TF-IDF, CounterVectorizor, TransactionEncoder도 같은 방식으로 메모리소요많고 오래걸린다.




In [12]:
#result에서 마지막 itemsets컬럼값 추출(기저귀, 쥬스, 우유)
## 쥬스, 우유와 함께 추천하면 좋을 item은??

In [34]:
choice = result.iloc[10,1]
choice

frozenset({'기저귀', '우유', '쥬스'})

In [31]:
list(items)[0]

'기저귀'

# word2vec 을 사용하여 추천시스템 해보자
## 자연어에서 특정한 단어를 중심으로 앞 뒤의 단어를 유추(예측)

```
# 코드로 형식 지정됨
```



In [35]:
#word2vec설치
!pip install gensim

In [36]:
from gensim.models import Word2Vec

In [37]:
doc = [
    'you say goodbye and I say hello', 
    'i say happy or he said white',
    'we say unhappy and angry black'
]

In [38]:
doc2 = [
    '나는 굿바이라고 했고, 너는 헬로우라고 했어.', 
    '나는 기쁘다고 했고, 너는 하얗게 슬프다고 했어.',
    '우리는 행복하지 않고, 블랙으로 슬퍼'
]

In [39]:
#문장을 토큰으로 끊어내자.
sentences = [ sentence.split(' ') for sentence in doc ]
#sentence한 번 for문 돌때마다 리스트sentence.split(' ') 나옴.총3개의 리스트가 나옴
#[처리내용 for one in 복수형태의자료구조] => 항상 결과는 리스트
sentences

[['you', 'say', 'goodbye', 'and', 'I', 'say', 'hello'],
 ['i', 'say', 'happy', 'or', 'he', 'said', 'white'],
 ['we', 'say', 'unhappy', 'and', 'angry', 'black']]

In [40]:
sentences2 = [ sentence.split(' ') for sentence in doc2 ]
sentences2

[['나는', '굿바이라고', '했고,', '너는', '헬로우라고', '했어.'],
 ['나는', '기쁘다고', '했고,', '너는', '하얗게', '슬프다고', '했어.'],
 ['우리는', '행복하지', '않고,', '블랙으로', '슬퍼']]

In [42]:
model = Word2Vec(window=1, min_count=1)
#특정한 단어의 앞뒤에 1번나온패턴
#window => 유추하는 계산을  할 때 특정한 단어의 앞뒤 몇개를 가지고 계산을 할 지의 단위
#min_count => 특정한 패턴의 출현 횟수

In [44]:
#영문 문장을 학습을 시켜보자.
model.build_vocab(sentences)

In [45]:
#특정 단어를 주고 앞 뒤 단어를 추론해보자.
model.wv.most_similar('say')#비지도학습(say에 대해 학습시킨거 아니니까!)
#say앞으로는 he가 나올확률 가장 높음.
#say뒤로는 hello가 나올확률 가장 높음
##따라서 문장을 쓰면 he say hello가 될 확률이 가장 높다.

[('he', 0.18314552307128906),
 ('or', 0.13936923444271088),
 ('said', 0.07459166646003723),
 ('white', 0.060143548995256424),
 ('angry', 0.028369128704071045),
 ('unhappy', 0.027612939476966858),
 ('you', 0.0027031265199184418),
 ('hello', -0.026689164340496063),
 ('we', -0.029307441785931587),
 ('goodbye', -0.03243057802319527)]

In [47]:
model.wv.most_similar('happy')

[('hello', 0.2032647430896759),
 ('I', 0.08445782959461212),
 ('you', 0.0641375258564949),
 ('or', 0.05771534517407417),
 ('white', 0.05154849588871002),
 ('and', 0.013006090186536312),
 ('i', -0.00713206734508276),
 ('we', -0.01683427393436432),
 ('black', -0.01731646992266178),
 ('angry', -0.03409711271524429)]

In [49]:
#한글로해보자.
model2 = Word2Vec(window=1, min_count=1)
model2.build_vocab(sentences2)

In [52]:
model2.wv.most_similar('너는')

[('행복하지', 0.15571533143520355),
 ('않고,', 0.09693466871976852),
 ('굿바이라고', 0.09317640215158463),
 ('나는', 0.031186997890472412),
 ('블랙으로', 0.028997167944908142),
 ('우리는', 0.02460627630352974),
 ('슬퍼', 0.02244904637336731),
 ('했어.', 0.0032214000821113586),
 ('슬프다고', -0.06080716848373413),
 ('하얗게', -0.09359260648488998)]

In [54]:
#doc, doc2에서 각 문장간의 유사도를 판단해보자.

## 1. stop_words로 불용어처리하기
## 2. 원핫인코딩으로 너무 자주 등장하는 단어 가중치 줄이기.
## 3. 나머지로 원핫인코딩 방식으로 메트릭스 생성.

In [57]:
tfidf = TfidfVectorizer(stop_words='english')#불용어처리가능한 ifidf객체 생성
#영어가 아닌것들 없애버리기
#ifidf는 빈도수를 구하긴 하지만, 문서마다 자주 등장하는 문서는 각 문서를 구분할 단어로
#사용하기는 부적합하여 가중치를 낮추어 계산.

In [58]:
tfidf_matrix = tfidf.fit_transform(doc)
tfidf_matrix

<3x9 sparse matrix of type '<class 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [59]:
# 3*9에서
# 3은 문서의 개수
# 9는 단어의 개수

In [60]:
tfidf_matrix.toarray()

array([[0.        , 0.        , 0.54270061, 0.        , 0.54270061,
        0.        , 0.64105545, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.54645401, 0.        ,
        0.54645401, 0.32274454, 0.        , 0.54645401],
       [0.54645401, 0.54645401, 0.        , 0.        , 0.        ,
        0.        , 0.32274454, 0.54645401, 0.        ]])

In [63]:
#문장간에 유사도를 구해봅시다

# cosine_similarity(tfidf_matrix, tfidf_matrix)의
# 행은 문서1, 문서2, 문서3
# 열도 문서1, 문서2, 문서3
# 문장 간 유사도는?
## 문서1+문서2:0.20689715
## 문서1+문서3:0.20689715
## 문서2+문서3:0.10416404
### 코사인 유사도는 표절검사할 때 쓰인다.

In [62]:
cosine_similarity(tfidf_matrix, tfidf_matrix)

array([[1.        , 0.20689715, 0.20689715],
       [0.20689715, 1.        , 0.10416404],
       [0.20689715, 0.10416404, 1.        ]])

In [64]:
#행은 문서1, 문서2, 문서3
#열도 문서1, 문서2, 문서3
#유사도는?

In [65]:
#장바구니 유사도를 구해보자.

# 장바구니 유사도 측정결과
## 장바구니1과 장바구니2는 33%정도 유사도가 있다.
## 장바구니1과 장바구니3는 87%정도 유사도가 있다.
## 장바구니1과 장바구니4는 유사도가 없다.
## 장바구니2과 장바구니3는 58%정도 유사도가 있다.
## 장바구니2과 장바구니4는 82%정도 유사도가 있다.
## 장바구니3과 장바구니4는 35%정도 유사도가 있다.
### 장바구니1과 유사도가 가장 높은 장바구니를 찾아서 추천해주고 싶은 item을 골라보자.
### => 장바구니1과 장바구니3이 가장 유사도가 높고 (약87%) '양상추'를 추천.
 

In [67]:
#2차행렬 T/F로 인코딩하고 바꿔준 te_ary객체를 이용해서 코사인 유사도 측정을 해보자.
#코사인유사도를 측정하려면 무조건 인코딩이 되어있어야함!!!!.
cosine_similarity(te_ary, te_ary)
#행렬 모두 장바구니1, 장바구니2, 장바구니3, 장바구니4

array([[1.        , 0.33333333, 0.8660254 , 0.        ],
       [0.33333333, 1.        , 0.57735027, 0.81649658],
       [0.8660254 , 0.57735027, 1.        , 0.35355339],
       [0.        , 0.81649658, 0.35355339, 1.        ]])